In [1]:
import pandas as pd
from epiweeks import Week
from bulletin.data.sivep import sivep

pd.set_option('display.max_columns', None)

In [2]:
sivep = sivep()
sivep.read_all_database_files()

In [3]:
sivep_df = sivep.get_df()

In [4]:
sivep_df = sivep_df.loc[sivep_df['SG_UF'] == 'PR']
sivep_df = sivep_df.loc[sivep_df['SG_UF_NOT'] == 'PR']
sivep_df = sivep_df.loc[sivep_df['AMOSTRA'] == '1']
sivep_df.loc[sivep_df['CLASSI_FIN'].isnull(), 'CLASSI_FIN'] = '0'

In [5]:
sivep_df['DT_NOTIFIC'] = sivep_df['DT_NOTIFIC'].apply(lambda x: pd.to_datetime(x, format='%d/%m/%Y'))
sivep_df['SE'] = sivep_df['DT_NOTIFIC'].apply(lambda x: Week.fromdate(x))

In [6]:
tabela = sivep_df.groupby(['SE','CLASSI_FIN'])[['NU_NOTIFIC']].count().unstack()

In [7]:
tabela = tabela.droplevel(0, axis=1)

In [8]:
# tabela['Semana Epidemiológica'] = [ x.week for x in tabela.index.values]
# tabela = tabela.set_index('Semana Epidemiológica')

In [9]:
tabela['1'] = tabela['1'].add(tabela['2'])
tabela['1'] = tabela['1'].add(tabela['3'])
tabela = tabela[['5', '1', '4', '0']]

In [10]:
tabela = tabela.rename(columns={'5':'Casos confirmados', '1':'Casos descartados', '4':'Casos suspeitos', '0':'Não informados'})

In [11]:
tabela['Total testes'] = tabela.sum(1)
tabela['Positividade (%)'] = tabela['Casos confirmados'].div(tabela['Total testes'])

In [12]:
tabela.to_excel('output/TABELA_testes_sivep.xlsx')

In [13]:
tabela

CLASSI_FIN,Casos confirmados,Casos descartados,Casos suspeitos,Não informados,Total testes,Positividade (%)
SE,,,,,,
202001,1.0,NaN,8.0,NaN,9.0,0.111111
202002,1.0,NaN,27.0,2.0,30.0,0.033333
202003,1.0,NaN,26.0,NaN,27.0,0.037037
202004,NaN,NaN,23.0,NaN,23.0,NaN
202005,NaN,17.0,32.0,NaN,49.0,NaN
...,...,...,...,...,...,...
202114,2072.0,23.0,358.0,767.0,3220.0,0.643478
202115,2000.0,26.0,310.0,965.0,3301.0,0.605877
202116,1513.0,35.0,196.0,1102.0,2846.0,0.531623
